In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('data/cars_datasets_2025.csv', encoding='latin-1')

In [3]:
pd.set_option('display.float_format', '{:,.2f}'.format)

In [4]:
df.head()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


View the last 5 rows

In [5]:
df.tail()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
1213,Toyota,Crown Signia,2.5L Hybrid I4,2487 cc,240 hp,180 km/h,7.6 sec,"$43,590  $48,000",Hybrid (Gas + Electric),5,239 Nm
1214,Toyota,4Runner (6th Gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326 hp,180 km/h,6.8 sec,"$50,000",Hybrid,7,630 Nm
1215,Toyota,Corolla Cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,169  196 hp,190 km/h,8.0  9.2 sec,"$25,210  $29,135",Gas / Hybrid,5,190  210 Nm
1216,Toyota,C-HR+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,140  198 hp,180 km/h,7.9  10.5 sec," 33,000",Hybrid,5,190  205 Nm
1217,Toyota,RAV4 (6th Gen),2.5L Hybrid / Plug-in Hybrid,2487 cc + batt,219  302 hp,200 km/h,6.0  8.1 sec,"$29,000  $43,000",Hybrid / Plug-in,5,221  400 Nm


View sample rows (n)

In [6]:
df.sample(20)

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
792,GMC,Yukon XL Denali Reserve,6.2L V8 engine,"6,162 cc",420 hp,210 km/h,6.0 sec,"$82,000",Petrol,7,624 Nm
933,Jaguar Land Rover,Jaguar I-PACE First Edition,Electric (Dual Motor),90 kWh (Battery),400 hp,200 km/h,4.5 sec,"$85,000",Electric,5,696 Nm
1028,Ford,F-350 Super Duty,6.7L Turbo-Diesel / 7.3L V8,"6,700 - 7,300 cc",450 - 475 hp,180 km/h,7.0 - 8.0 sec,"$45,000 - $75,000",Diesel,5,"1,050 - 1,200 Nm"
201,TOYOTA,PRIUS,I4,"1,798 cc",121 hp,180 km/h,10.5 sec,"$26,700",Hybrid,5,142 Nm
592,Porsche,Cayenne Platinum Edition,3.0L Turbocharged V6,2995 cc,335 hp,245 km/h,5.8,"$79,000",Petrol,5,450 Nm
285,Volkswagen,Caddy,1.5L Turbo I4,1498 cc,102 - 150 hp,185 km/h,10.5 sec,"$28,000","Petrol, Diesel",7,250 Nm
644,Tata Motors,Manza,1.3L Quadrajet Diesel / 1.4L Petrol,1248 cc / 1368 cc,75 hp / 90 hp,160 km/h,14 sec,"$9,000",Diesel/Petrol,5,200 Nm / 115 Nm
914,Jaguar Land Rover,Jaguar XJ LWB (Long Wheelbase),V6 Petrol,2995 cc (V6),300 hp,250 km/h,6.0 sec,"$90,000",Petrol,5,400 Nm
464,Nissan,Murano CrossCabriolet,3.5L V6,3498 cc,260 HP,195 km/h,7.6 sec,"$47,000",Petrol,4,325 Nm
1214,Toyota,4Runner (6th Gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326 hp,180 km/h,6.8 sec,"$50,000",Hybrid,7,630 Nm


Check for empty columns

In [11]:
df.isnull().sum()

Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          0
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    0
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       0
dtype: int64

Inspect Empty Columns

In [8]:
empty_columns = df[df.isna().any(axis=1)]
print(empty_columns.to_string(index=False))

Company Names             Cars Names            Engines CC/Battery Capacity    HorsePower     Total Speed Performance(0 - 100 )KM/H        Cars Prices      Fuel Types Seats Torque
     MERCEDES            BENZ EQS 53     ELECTRIC MOTOR                 NaN        751 hp        250 km/h                   3.4 sec          $102,000         Electric     5 950 Nm
       TOYOTA                COASTER        4.0L,DIESEL            4,009 cc        134 hp        120 km/h                       NaN           $70,000           Diesel    20 500 Nm
       Nissan                  Urvan   2.5L Turbo Diese                 NaN       2488 cc        140 km/h                  14.0 sec           $28,000           Diesel    15 356 Nm
   Mitsubishi         Canter (Truck)        3.0L Diesel            3,000 cc        150 hp        120 km/h                       NaN  $35,000 - $45,000          Diesel     2 370 Nm
   Mitsubishi   Fuso Fighter (Truck)        7.5L Diesel            7,500 cc        240 hp        120

Drop empty columns

In [10]:
df = df.drop(empty_columns.index)

Check for duplicates

In [15]:
df.duplicated().sum()
df[df.duplicated()]

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque


In [13]:
duplicates = df[df.duplicated(keep=False)]
print(duplicates.to_string(index=False))

Company Names     Cars Names                        Engines CC/Battery Capacity   HorsePower Total Speed Performance(0 - 100 )KM/H Cars Prices Fuel Types Seats Torque
   Volkswagen Golf Cabriolet  1.2L I4 Turbo / 2.0L I4 Turbo   1197 cc / 1984 cc 105 - 210 hp    217 km/h                   8.4 sec    $30,000      Petrol     4 200 Nm
   Volkswagen Golf Cabriolet  1.2L I4 Turbo / 2.0L I4 Turbo   1197 cc / 1984 cc 105 - 210 hp    217 km/h                   8.4 sec    $30,000      Petrol     4 200 Nm
   Volkswagen   Jetta Hybrid 1.4L I4 Turbo + Electric Motor             1395 cc       170 hp    210 km/h                   8.6 sec    $28,000      Hybrid     5 250 Nm
   Volkswagen   Jetta Hybrid 1.4L I4 Turbo + Electric Motor             1395 cc       170 hp    210 km/h                   8.6 sec    $28,000      Hybrid     5 250 Nm
  Tata Motors       Tiago EV   Permanent Magnet Synchronous              24 kWh        74 hp    120 km/h                   5.7 sec    $11,000    Electric     5 114 N

Drop Duplicates

In [14]:
df = df.drop_duplicates(keep="first")

Data summary

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1206 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1206 non-null   object
 1   Cars Names                 1206 non-null   object
 2   Engines                    1206 non-null   object
 3   CC/Battery Capacity        1206 non-null   object
 4   HorsePower                 1206 non-null   object
 5   Total Speed                1206 non-null   object
 6   Performance(0 - 100 )KM/H  1206 non-null   object
 7   Cars Prices                1206 non-null   object
 8   Fuel Types                 1206 non-null   object
 9   Seats                      1206 non-null   object
 10  Torque                     1206 non-null   object
dtypes: object(11)
memory usage: 113.1+ KB


Column summary statistics

In [16]:
df.describe()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
count,1206,1206,1206,1206,1206,1206,1206,1206,1206,1206,1206
unique,37,1193,350,309,453,113,180,533,21,17,262
top,Nissan,Panamera Platinum Edition,I4,"2,000 cc",310 hp,250 km/h,6.5 sec,"$35,000",Petrol,5,400 Nm
freq,148,2,64,31,22,144,45,36,869,689,72



Rename columns for easier referencing

In [18]:
print(df.columns)

new_columns = ['car_brand', 'car_name', 'engine', 'cc_battery_capacity', 'horsepower', 'total_speed_kmh', '0_100_kmh_performance_secs', 'price', 'fuel_type', 'seats', 'torque_nm']

df.columns = new_columns

print(df.columns)

Index(['Company Names', 'Cars Names', 'Engines', 'CC/Battery Capacity',
       'HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices',
       'Fuel Types', 'Seats', 'Torque'],
      dtype='object')
Index(['car_brand', 'car_name', 'engine', 'cc_battery_capacity', 'horsepower',
       'total_speed_kmh', '0_100_kmh_performance_secs', 'price', 'fuel_type',
       'seats', 'torque_nm'],
      dtype='object')


Split cars with multiple engine types into different rows

In [20]:
df = df.reset_index(names="og_id")

cc_re = re.compile(r'([\d.,]+)\s*cc\b', flags=re.I) # find cc values
kwh_range = re.compile(r'([\d.,]+)\s*[-–]\s*([\d.,]+)\s*kwh\b', re.I) # match kwh ranges: 'x-y kwh'
kwh_re = re.compile(r'([\d.,]+)\s*kwh\b', flags=re.I) # match single kwh value: 'x kwh'
num_re = re.compile(r'([\d.,]+)') # any number that hasn't already been matched
paren_re = re.compile(r'\([^)]*\)') 
iv_token = re.compile(r'\b[ivx]\s*\d\b', re.I)

general_norm = re.compile(r'\s*(?:/|,(?=\s*[A-Za-z])|-(?!\s*\d+\s*kwh))\s*', re.I)
engine_norm = re.compile(r'\s*/\s*', re.I)
fuel_norm = re.compile(r'\s*(?:/|,(?=\s*[A-Za-z]))\s*', re.I) # ensure fuel doesn't get split by dashes

cols_to_split = [
    "engine",
    "cc_capacity",
    "kwh_capacity",
    "horsepower",
    "0_100_kmh_performance_secs",
    "fuel_type",
    "torque_nm"
]

# remove commas from regex matches and turn to float
def _nums(s):
    out = []
    for m in s:
        try:
            out.append(float(m.replace(',', '')))
        except Exception:
            pass
    return out

def extract_capacity_lists(val):
    s = str(val)
    s_lower = s.lower()
    s_norm = kwh_range.sub(r'\1 kwh / \2 kwh', s_lower) # turn kwh ranges from 'x-y kwh' to 'x kwh' / 'y kwh' for easier splitting later

    # place kwh values into a list
    kwh_nums = _nums(kwh_re.findall(s_norm))
    kwh_list = kwh_nums if kwh_nums else [pd.NA]
    
    # prepare non-kwh matches
    no_kwh = kwh_re.sub("", s_norm)
    no_kwh = paren_re.sub(" ", no_kwh) # strip parenthesis its contents
    no_kwh = iv_token.sub(" ", no_kwh) # strip any i or v matches

    cc_nums = _nums(cc_re.findall(no_kwh)) # find cc values
    no_kwh_cc = cc_re.sub("", no_kwh) # non-kwh/cc matches

    cc_fallback = _nums(num_re.findall(no_kwh_cc)) # if no cc or kwh attached to value, assume it's cc
    cc_all = (cc_nums + cc_fallback) if (cc_nums or cc_fallback) else [pd.NA]

    return pd.Series([cc_all, kwh_list])

def pre_engine(s):
    if pd.isna(s):
        return s
    t = str(s)
    t = re.sub(r'\b(?:or|vs)\b', ' / ', t, flags=re.I) # standardize splitting engines with '/' instead of 'or' or 'vs'
    t = re.sub(r'(?<!\d)\s*,\s*(?!\d)', ' ', t) # ignores commas in numbers
    t = re.sub(r'\binline\s*[- ]?\s*(\d)\b', r'inline-\1', t, flags=re.I) # standardize 'inline 4' as 'inline-4'
    t = re.sub(r'\bi\s*[- ]?\s*(\d)\b', r'i\1', t, flags=re.I) # standardize 'i 4' as 'i4'
    t = re.sub(r'\bv\s*[- ]?\s*(\d)\b', r'v\1', t, flags=re.I) # standardize 'v 6' as 'v6'
    t = re.sub(r'\s*/\s*', ' / ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

# normalize splitting rows with '/'
def normalize_and_split(x, norm):
    if isinstance(x, list):
        return x
    s = re.sub(norm, '/', str(x)) # replace separators with '/'
    parts = [p.strip() for p in s.split('/') if p.strip()] # strip then split on '/'
    return parts if parts else [pd.NA]

def _clean_parts(lst):
    if not isinstance(lst, list):
        return lst
    return [re.sub(r'\s*,\s*$', '', p).strip() for p in lst]

# create cc and kwh capacity columns from cc_batter_capacity column
df[["cc_capacity", "kwh_capacity"]] = df["cc_battery_capacity"].apply(extract_capacity_lists)
df = df.drop(columns=["cc_battery_capacity"]) # drop original column

df["engine"] = df["engine"].map(pre_engine)
df["engine"] = df["engine"].map(lambda v: normalize_and_split(v, engine_norm))
df["engine"] = df["engine"].map(_clean_parts)

for col in cols_to_split:
    if col in ("cc_capacity", "kwh_capacity", "engine"):
        continue
    elif col == "fuel_type":
        pat = fuel_norm
    else:
        pat = general_norm
    df[col] = df[col].map(lambda v: normalize_and_split(v, pat))

df["_max_splits"] = df[cols_to_split].apply(lambda r: max(len(r[c]) for c in cols_to_split), axis=1) # determine how many rows to make
df = df.loc[df.index.repeat(df["_max_splits"].where(df["_max_splits"] > 1, 1))].copy()
df["_idx"] = df.groupby("og_id").cumcount()
df["_idx"] = pd.to_numeric(df["_idx"], errors="coerce").fillna(-1).astype("Int64")

def take_n_request(row, col):
    vals = row[col]
    j = min(row["_idx"], len(vals) - 1)
    return vals[j]

for col in cols_to_split:
    df[col] = df.apply(take_n_request, axis=1, col=col) #???

# create row_id column consisting of "og_id" and "_idx" to highlight highlight original row and what variant it is
df["row_id"] = df["og_id"].astype(str) + "-" + df["_idx"].astype(str)
df.insert(0, "row_id", df.pop("row_id"))

df = df.drop(columns=["_max_splits"])

Address odd characters

In [21]:
def scrub_ascii(x):
    if isinstance(x, str):
        return re.sub(r'[^\x00-\x7F]+', '-', x)
    return x

obj_cols = df.select_dtypes(include="object").columns
for col in obj_cols:
    df[col] = df[col].map(scrub_ascii)

Remove non-numerical characters in numerical columns

In [22]:
df['price'] = (df['price'].str.replace('$', '', case=False).str.replace(',', '', case=False)).str.strip()

numeric_cols = ["horsepower", "torque_nm", "0_100_kmh_performance_secs", "total_speed_kmh"]

for col in numeric_cols:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(r'(\d)\.\s+(\d)', r'\1.\2', regex=True)
    df[col] = df[col].str.replace(r'[A-Za-z/ ]+', '', regex=True)
    df[col] = df[col].str.replace(r'[^0-9\.\-]', '', regex=True)
    df[col] = df[col].str.strip()

Address range columns

In [23]:
for col in ["price", "horsepower", "torque_nm", "0_100_kmh_performance_secs"]:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.strip()
    
range_re = re.compile(r'-?\d+(?:\.\d+)?')

def extract_range(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    
    s = str(val)
    nums = range_re.findall(s)
    nums = [float(n) for n in nums] 
    
    if len(nums) == 0:
        return pd.Series([np.nan, np.nan])
    elif len(nums) == 1:
        return pd.Series([nums[0], nums[0]])
    else:
        a, b = nums[0], nums[1]
        return pd.Series([min(a, b), max(a, b)])
    
def extract_range_performance(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    
    s = str(val)
    nums = range_re.findall(s)
    nums = [float(n) for n in nums]

    if len(nums) == 0:
        return pd.Series([np.nan, np.nan])
    
    realistic = [n for n in nums if n > 0]

    if len(realistic) == 0:
        realistic = nums

    if len(realistic) == 1:
        return pd.Series([realistic[0], realistic[0]])
    else:
        a, b = realistic[0], realistic[1]
        return pd.Series([min(a, b), max(a, b)])
    
for col in ["price", "horsepower", "torque_nm"]:
    new_cols = [f"{col}_min", f"{col}_max"]
    df[new_cols] = df[col].apply(extract_range)

df[["0_100_kmh_performance_secs_min",
    "0_100_kmh_performance_secs_max"]] = (
        df["0_100_kmh_performance_secs"].apply(extract_range_performance)
    )

df = df.drop(columns=["price", "horsepower", "torque_nm", "0_100_kmh_performance_secs"])

Address Seats Column

In [24]:
def format_seats(val):
    if pd.isna(val):
        return pd.Series([np.nan, np.nan])
    
    s = str(val).strip()
    if re.fullmatch(r'\d+', s):
        n = int(s)
        return pd.Series([n,n])
    
    add_seats = re.fullmatch(r'(\d+)\+(\d+)', s)
    if add_seats:
        a, b = int(add_seats.group(1)), int(add_seats.group(2))
        total = a + b
        return pd.Series([total, total])
    
    range_seats = re.fullmatch(r'(\d+)\s*-\s*(\d+)', s)
    if range_seats:
        a, b = int(range_seats.group(1)), int(range_seats.group(2))
        return pd.Series([min(a, b), max(a, b)])
    
    return pd.Series([np.nan, np.nan])

df[['seats_min', 'seats_max']] = df['seats'].apply(format_seats)
df = df.drop(columns=["seats"])

Convert columns to numerical type

In [25]:
numeric_cols = [
    "total_speed_kmh", 
    "seats_min", "seats_max",
    "cc_capacity", "kwh_capacity",
    "price_min", "price_max",
    "horsepower_min", "horsepower_max",
    "torque_nm_min", "torque_nm_max",
    "0_100_kmh_performance_secs_min", "0_100_kmh_performance_secs_max"
]

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

Check Fuel-Type Categories

In [30]:
df['fuel_type'].value_counts()

fuel_type
petrol      980
diesel      143
hybrid      112
electric    103
Name: count, dtype: int64

Normalize Fuel Type Categories

In [27]:
df['fuel_type'] = df['fuel_type'].str.lower().str.strip()

df['fuel_type'] = df['fuel_type'].replace({
    'electric': 'electric',
    'ev': 'electric',
    'plug-in': 'electric',
    'hybrid': 'hybrid',
    'plug in hybrid': 'hybrid',
    'plug-in hybrid': 'hybrid',
    'plug in hyrbrid': 'hybrid',
    'hybrid (gas + electric)': 'hybrid',
    'hybrid (petrol)': 'hybrid',
    'petrol': 'petrol',
    'gas': 'petrol',
    'awd': 'petrol',
    'diesel': 'diesel',
    'hydrogen': 'hydrogen',
    'cng': 'cng',
    
})

Drop CNG and Hydrogen Rows

In [29]:
# only 3 hydrogen and 1 CNG row, not enough to warrant having its own fuel type category so I thought it was best to omit it
drop_fuel_types = df[df['fuel_type'].str.lower().isin(['cng', 'hydrogen'])]

df = df.drop(drop_fuel_types.index)

Check Car Brands

In [33]:
df['car_brand'].unique()

array(['Ferrari', 'Rolls Royce', 'Ford', 'Mercedes', 'Audi', 'Bmw',
       'Aston Martin', 'Bentley', 'Lamborghini', 'Toyota', 'Nissan',
       'Volvo', 'Kia', 'Honda', 'Hyundai', 'Mahindra', 'Maruti Suzuki',
       'Volkswagen', 'Porsche', 'Cadillac', 'Tata Motors', 'Tesla',
       'Jeep', 'Mazda', 'Chevrolet', 'Gmc', 'Peugeot', 'Bugatti',
       'Jaguar Land Rover', 'Acura', 'Mitsubishi'], dtype=object)

Fix Capitalization

In [32]:
df['car_brand'] = (
    df['car_brand']
    .str.strip()
    .str.lower()
    .str.title()
)

In [34]:
region_map = {
'Volkswagen': 'Europe', 'Porsche': 'Europe', 'Peugeot': 'Europe', 'Jaguar Land Rover': 'Europe', 'Bmw': 'Europe', 'Lamborghini': 'Europe', 'Audi': 'Europe', 'Mercedes': 'Europe', 'Rolls Royce': 'Europe', 'Volvo': 'Europe','Aston Martin': 'Europe', 'Bugatti': 'Europe', 'Ferrari': 'Europe', 'Bentley': 'Europe',
'Nissan': 'Asia', 'Mazda': 'Asia', 'Kia': 'Asia', 'Toyota': 'Asia', 'Mitsubishi': 'Asia', 'Hyundai': 'Asia', 'Acura': 'Asia', 'Honda': 'Asia', 'Maruti Suzuki': 'Asia', 'Tata Motors': 'Asia', 'Mahindra': 'Asia',
'Ford': 'North America', 'Gmc': 'North America', 'Chevrolet': 'North America', 'Tesla': 'North America', 'Cadillac': 'North America', 'Jeep': 'North America'
}

df['region'] = df['car_brand'].map(region_map).fillna('Other')

In [35]:
country_map = {
    'Ferrari': 'Italy',
    'Rolls Royce': 'United Kingdom',
    'Ford': 'United States',
    'Mercedes': 'Germany',
    'Audi': 'Germany',
    'Bmw': 'Germany',
    'Aston Martin': 'United Kingdom',
    'Bentley': 'United Kingdom',
    'Lamborghini': 'Italy',
    'Toyota': 'Japan',
    'Nissan': 'Japan',
    'Volvo': 'Sweden',
    'Kia': 'South Korea',
    'Honda': 'Japan',
    'Hyundai': 'South Korea',
    'Mahindra': 'India',
    'Maruti Suzuki': 'India',
    'Volkswagen': 'Germany',
    'Porsche': 'Germany',
    'Cadillac': 'United States',
    'Tata Motors': 'India',
    'Tesla': 'United States',
    'Jeep': 'United States',
    'Mazda': 'Japan',
    'Chevrolet': 'United States',
    'Gmc': 'United States',
    'Peugeot': 'France',
    'Bugatti': 'France',
    'Jaguar Land Rover': 'United Kingdom',
    'Acura': 'Japan',
    'Mitsubishi': 'Japan'
}

df['country'] = df['car_brand'].map(country_map).fillna('Other')

In [37]:
bins = [0, 25000, 50000, 100000, 250000, float('inf')]
labels = ['Budget', 'Midrange', 'Premium', 'Luxury', 'Supercar']

df['price_tier'] = pd.cut(df['price_max'], bins=bins, labels=labels, right=True)

df.groupby('price_tier', observed=True).agg(
    avg_price=('price_max', 'mean'),
    avg_hp=('horsepower_max', 'mean'),
    avg_performance=('0_100_kmh_performance_secs_min', 'mean'),
    avg_torque=('torque_nm_max', 'mean'),
    avg_cc_capacity=('cc_capacity', 'mean'),
    avg_total_speed=('total_speed_kmh', 'mean'),
    avg_seats=('seats_max','mean'),
    diesel=('fuel_type', lambda x: (x.str.lower() == 'diesel').sum()),
    petrol=('fuel_type', lambda x: (x.str.lower() == 'petrol').sum()),
    hybrid=('fuel_type', lambda x: (x.str.lower() == 'hybrid').sum()),
    electric=('fuel_type', lambda x: (x.str.lower() == 'electric').sum()),
    europe=('region', lambda x: (x.str.lower() == 'europe').sum()),
    asia=('region', lambda x: (x.str.lower() == 'asia').sum()),
    america=('region', lambda x: (x.str.lower() == 'north america').sum()),
    car_count=('price_max', 'count')
)

,avg_price,avg_hp,avg_performance,avg_torque,avg_cc_capacity,avg_total_speed,avg_seats,diesel,petrol,hybrid,electric,europe,asia,america,car_count
price_tier,,,,,,,,,,,,,,,
Budget,"18,804.80",116.94,11.27,179.98,"1,590.86",175.75,4.87,35,228,11,8,57,208,17,282
Midrange,"37,337.37",218.01,7.96,339.63,"2,338.46",204.20,5.20,75,371,66,34,183,260,103,546
Premium,"68,648.73",373.70,6.18,658.00,"3,989.84",221.31,5.31,28,223,15,44,136,65,109,310
Luxury,"152,348.74",559.20,4.29,"1,246.49","4,333.63",276.20,3.81,5,78,15,17,93,11,11,115
Supercar,"1,287,476.35",760.72,3.63,866.25,"5,990.51",315.06,3.04,0,80,5,0,80,4,1,85


In [38]:
df.to_csv("cars_dataset_2025_cleaned.csv", index=False)